In [ ]:
import numpy as np
from math import tan, pi
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib import cm 

In [ ]:
# load tomography and iasp model
data_korean = np.loadtxt("Koreavel.xyz", skiprows=1)
deep    = data_korean[:,2]
d = deep >= 40
P_data_anonmal = data_korean[d,3] - data_korean[d,4] # delta Vp 
S_data_anonmal = data_korean[d,5] - data_korean[d,6] # delta Vs
S_data = data_korean[d,6]
adiabat = np.loadtxt('adiabat_pt.txt') # adiabatic temp from burnman
T_ref = np.loadtxt('reference_temp_c.txt', skiprows=1)
longitude = data_korean[d,0]
latitude = data_korean[d,1]
depth    = data_korean[d,2]

In [ ]:
print (len(np.unique(depth)), len(np.unique(latitude)), len(np.unique(longitude)))

In [ ]:
# define mineral physics constants used in this study
## The table is from Fabio et al., 2003
data = np.array([('ol', 0.82, 4.2, 1.4, -0.017e9, -0.014e9, 129e9, 81e9, 3222, 0.201e-4, 0.139e-7, 0.1627e-2, 
                  1.658e2, 0.2332e-2, -0.3971e7) , 
       ('opx', 0.144, 7, 1.6, -0.027e9, -0.012e9, 109e9, 75e9, 3215, 0.3871e-4, 0.0446e-7, 0.03435e-2, 
       1.855e2, 0.233e-2, -0.6326e7) , 
       ('gt', 0.036, 4.4, 1.4, -0.019e9, -0.01e9, 171e9, 92e9, 3565, 0.0991e-4, 0.1165e-7, 1.0624e-2, 
       1.44e2, 0.188e-2, -0.135e7)], 
        dtype=[('name', 'U10'), ('conc', 'f4'), ('KPDer', 'f4'), ('MuPDer', 'f4'), 
        ('KTDer', 'f4'), ('MuTDer', 'f4'), ('K', 'f4'), ('Mu', 'f4'), ('rho', 'f4'), 
        ('a0', 'f4'), ('a1', 'f4'), ('a2', 'f4'), ('cp0', 'f4'), ('cp1', 'f4'), ('cp2', 'f4')])

## Anelasticity Parameter 
## There is original reference in Goes and Govers, 2000, Shallow mantle temperatures under Europe from P and S
## wave tomography, JGR, 97
A = 1.48e-1 ; H = 500e3; V = 20e-6; a = 0.15 ;d=1e-3
#A = 0.0013e-6
#E = 424e3
R = 8.314
grad_crust = -0.17; ## Yan et al., 2016, Stress development in heterogenetic lithosphere. Techtonophysics, 671, pp 56-62.
grad_crust2 = -0.25; ## Yan et al., 2016, Stress development in heterogenetic lithosphere. Techtonophysics, 671, pp 56-62.

We assume that for a fixed pressure $p_{0}$, density at temperature $T$ is determined with respect to a reference value at $T_{0}$ as follows:
\begin{equation}
\rho(T, P_0) = \rho(T_0, P_0)\exp\left( -\int_{T_0}^{T} \alpha(T^{\prime})\ dT^{\prime} \right), \qquad \text{(eq.1)}
\end{equation}
where $\alpha$ is thermal expansion coefficient. 

Following [Saxena and Shen, 1992, JGR, 97, pp 19813-19825], the polynomial form of $\alpha$ is assumed to be
\begin{equation}
\alpha(T)=\alpha_0 + \alpha_1T + \alpha_2T^{-1} + \alpha_3T^{-2} + \ldots.
\end{equation}

Plugging the polynomial form into eq. 1, we get
\begin{equation}
\rho(T, P_0) = \rho(T_0, P_0) \exp \left(-(\alpha_0(T-T_0) + \frac{1}{2}\alpha_1(T-T_0)^2 + \alpha_2ln(T-T_0) + ...) \right). \qquad \text{(eq.2)}
\end{equation}

**[EC]**: I think we should get the following as eq.2:
\begin{equation}
\rho(T, P_0) = \rho(T_0, P_0) \exp \left(-\left[\alpha_{0}\,(T-T_0) + \frac{1}{2}\alpha_{1}\,(T^{2}-T_{0}^{2}) + \alpha_2\,(\ln(T)-\ln(T_{0})) + \ldots \right] \right). \qquad \text{(eq.2)}
\end{equation}

### Notes from the code block for density function:
- We should check what T0 means (atmophere temperature or surface temperautre)
- In equation 1, T0 means 293 K (i.e. atmospheric temperature) 
- Equation 1 means that density changes in temperature using polynomoal thermal expansion with depth

In [ ]:
def density(a0, a1, a2, rho0, T):
    T0 = 293.0
    #return (rho0 * np.exp (-a0 * (1600 - 293) - ( a1*(1600 - 293)**2 )/2 - a2*np.log (1600 - 293)))
    #return (rho0 * np.exp (-a0 * (T - T0) - ( a1*(T - T0)**2 )/2 - a2*np.log (T - T0))) 
    return (rho0 * np.exp (-a0 * (T - T0) - 0.5*a1*(T**2 - T0**2) - a2*(np.log(T)-np.log(T0)))) 

**[EC]** We need the source for the equation, explanation for all the parameters used and the equation itself.

<!--$$P + (1+\epsilon)^{5/2}(3K\epsilon+0.5(9K(4-K')\epsilon^2 + ...)= 0 \dots (eq.3)$$-->
\begin{equation}
P + (1+\epsilon)^{5/2}\, \left( 3K\epsilon + \frac{1}{2}\,9K(4-K')\,\epsilon^2 + \ldots \right) = 0 \qquad \text{(eq.3)}
\end{equation}

In [ ]:
def derivative (f, x, h, P, K, Kdash):
    return (f(x+h, P, K, Kdash) - f(x-h, P, K, Kdash)) / (2.0*h) # might want to return a small non-zero if ==0

def epsilon(x, P, K, Kdash):
    return  ( (1 - 2*x) ** (5/2) ) * (3 * K * x + ( ( 9 * K * (4 - Kdash) )/ 2 ) * (x ** 2) )  + P 

def solve_ep (f, x0, h, P, K, Kdash):
    lastX = x0
    nextX = lastX + 10 * h  # "different than lastX so loop starts OK
    #print(abs(lastX - nextX)) 
    while (abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()
        newY  = epsilon(nextX,  P, K, Kdash)                     # just for debug... see what happens
#         print (newY)     # print out progress... again just debug
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h, P, K, Kdash)  # update estimate using N-R
    return nextX

**[EC]** We need the source for the equation, explanation for all the parameters used and the equation itself.

$$\rho(P, T) = (1-2\epsilon)^{3/2}\rho(P_0, T) \dots (eq.4)$$

In [ ]:
x0 = 1.e-8
pressures = adiabat[:,0]
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
#T_a = adiabat[:,2]
T_r = T_ref[:,1] # reference temp with depth
strain = np.ones(len(pressures))
rho = np.ones(len(pressures))

# compositional dependence 
rho_avg = data['rho'][0] * data['conc'][0] + data['rho'][1] * data['conc'][1] + \
            data['conc'][2] * data['rho'][2]
K_avg = ( data['conc'][0] * data['K'][0] + data['conc'][1] * data['K'][1] + data['conc'][2]*data['K'][2])
Muavg = ( data['conc'][0] * data['Mu'][0] + data['conc'][1] * data['Mu'][1] + data['conc'][2]*data['Mu'][2])
KPDer_avg = ( data['conc'][0] * data['KPDer'][0] + data['conc'][1] * data['KPDer'][1] + \
          data['conc'][2]*data['KPDer'][2])

for i in range (len(pressures)):
#    Ta   = (T_a [ depth_a <= depth[i]].max()) # reference temperature with depth
    strain[i]     = solve_ep (epsilon, x0, 1.e-15, pressures[i], K_avg, KPDer_avg) # equation to get x
#     rho[i] = ( data['conc'][0] *  data['rho'][0] + data['conc'][1] * data['rho'][1] + \
#               data['conc'][2] * data['rho'][2] ) * ( ( 1 - ( 2 * strain[i] ) ) ** (3/2) )
    rho[i] = (( data['conc'][0] *  density ( data['a0'][0], data['a1'][0], data['a2'][0], data['rho'][0], T_r[i]) ) 
    + ( data['conc'][1] *  density ( data['a0'][1], data['a1'][1], data['a2'][1], data['rho'][1], T_r[i]) )
    + ( data['conc'][2] *  density ( data['a0'][2], data['a1'][2], data['a2'][2], data['rho'][2], T_r[i]) ) ) \
    * ( ( 1 - ( 2 * strain[i] ) ) ** (3/2) )
    
### Equation 3 means that density changes in pressure using polynomoal strain equation
### So, we can calculate density (P,T) at specific point

In [ ]:
print (rho)
print(len(rho))

$$ M = M(P_0,T_0)+\left(\left(\frac{\partial K}{\partial T}\right )_P+4/3\left(\frac{\partial G}{\partial T}\right )_P \right)(T-T_0) \dots (eq.5) $$

$$ \partial M = \left(\left(\frac{\partial K}{\partial T}\right )_P+4/3\left(\frac{\partial G}{\partial T}\right )_P \right)(T-T_0) \dots (eq.6) $$

In [ ]:
# corrections to the moduli with temperature
def moduliT (data, x, comp):
#    return  ( data['K'][comp] + (4./3)*data['Mu'][comp] + data['KTDer'][comp] * (1600 - 293) + (4./3) * data['MuTDer'][comp] * (1600 - 293) )
    return  ( data['K'][comp] + (4./3)*data['Mu'][comp] + data['KTDer'][comp] * (x - 293) + (4./3) * data['MuTDer'][comp] * (x - 293) )
    
### By definition of eq. 1, T0 is surface temp.

In [ ]:
# corrections to the moduli
def moduli ( To, data, x, comp):
    return  ( data['KTDer'][comp] * (x - To) + (4./3) * data['MuTDer'][comp] * (x - To) )

$$ G = G(P_0,T_0)+\left(\frac{\partial G}{\partial T}\right )_P(T-T_0) \dots (eq.7)$$

$$ \partial G = \left(\frac{\partial G}{\partial T}\right )_P(T-T_0) \dots (eq.8)$$

In [ ]:
# corrections to the moduli with temperature
def s_moduliT (data, x, comp):
#    return  ( data['Mu'][comp] + data['MuTDer'][comp] * (1600 - 293) )    
    return  ( data['Mu'][comp] + data['MuTDer'][comp] * (x - 293) )    

### By definition of eq. 1, T0 is surface temp.

In [ ]:
# corrections to the moduli
def s_moduli ( To, data, x, comp):
    return  ( data['MuTDer'][comp] * (x - To) )

$$ M'(T) = \left(\left(\frac{\partial K}{\partial P}\right )_T+4/3\left(\frac{\partial G}{\partial P}\right )_T \right)\exp(\int\limits_{T_0}^T \alpha(T')\ dT') = $$

$$\left( K'+ 4G'/3 \right)\exp(\alpha_0(T-T_0) + 0.5\alpha_1(T-T_0)^2 + \alpha_2ln(T-T_0) + ...) \dots (eq.9) $$

$$ \partial M'(T)/\partial T' = \left(\left(\frac{\partial K}{\partial P}\right )_T+4/3\left(\frac{\partial G}{\partial P}\right )_T \right)\exp(\int\limits_{T_0}^T \alpha(T')\ dT') = $$

$$\left( K'+ 4G'/3 \right)\exp(\alpha_0(T-T_0) + 0.5\alpha_1(T-T_0)^2 + \alpha_2ln(T-T_0) + ...)\alpha(T') \dots (eq.10) $$

$$ \partial M'(T) = \left(\left(\frac{\partial K}{\partial P}\right )_T+4/3\left(\frac{\partial G}{\partial P}\right )_T \right)\exp(\int\limits_{T_0}^T \alpha(T')\ dT') = $$

$$\left( K'+ 4G'/3 \right)\exp(\alpha_0(T-T_0) + 0.5\alpha_1(T-T_0)^2 + \alpha_2ln(T-T_0) + ...)\alpha(T')(T-T_r) \dots (eq.10b) $$

In [ ]:
# Modulus derivative to pressure of function of temperature 
def moduli_der0 (data, x, comp):
    return ( data['KPDer'][comp] + (4/3) * data['MuPDer'][comp] ) *   \
( np.exp ( data['a0'][comp] * (x - 293) + ( data['a1'][comp] / 2 ) * (x - 293) ** 2  \
           ) ) 

In [ ]:
def moduli_der (To, data, x, comp):
    return ( data['KPDer'][comp] + (4/3) * data['MuPDer'][comp] ) *   \
( ( np.exp ( data['a0'][comp] * (x - 300) + ( data['a1'][comp] / 2 ) * (x - 293) ** 2  \
           ) ) * ( data['a0'][comp] + data['a1'][comp] * (x) )  * ( x - To ) ) 


In [ ]:
# Modulus derivative to pressure of function of temperature 
def smoduli_der0 (data, x, comp):
    return ( data['MuPDer'][comp] ) *   \
(   np.exp ( data['a0'][comp] * (x - 293) + ( data['a1'][comp] / 2 ) * (x - 293) ** 2 + 
             data['a2'][comp]*np.log (x - 293)  \
           )  ) 

In [ ]:
def smoduli_der (To, data, x, comp):
    return ( data['MuPDer'][comp] ) *   \
( ( np.exp ( data['a0'][comp] * (x - 293) + ( data['a1'][comp] / 2 ) * (x - 293) ** 2  \
           ) ) * ( data['a0'][comp] + data['a1'][comp] * (x) )  * ( x - To ) ) 

### By definition of eq. 1, T0 is surface temp.
### However, eq. 7 is resulant of derivatives to temperaure difference 
### So, To means reference temp and X means perturbation temp

$$ M = (1-2\epsilon)^{5/2}\left\{M + \epsilon [5M-3KM'(T)] \right\}  \dots (eq.11)$$

$$\partial M = (1-2\epsilon)^{5/2}\left\{\frac{\partial M}{\partial T}(T-T_r)+ \epsilon [5\frac{\partial M}{\partial T}(T-T_r)-3\frac{\partial K}{\partial T} \left ( \frac{\partial K}{\partial P} -(4/3) \frac{\partial G}{\partial P}\right )(T-T_r) - 3K\partial M'(T)] \right\}  \dots (eq.12)$$

In [ ]:
# P wave Modulus (P,T)
def p_modulus0 (data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( moduliT(data, x, comp)   +   \
            ep * ( 5 * moduliT(data, x, comp)  - 3 * ( data['K'][comp] ) *  \
                 ( moduli_der0 (data, x, comp ) ) ) ) 

In [ ]:
# function for K and mu
def p_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( moduli ( To, data, x, comp)   +   \
            ep * ( 5 * moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * ( data['KPDer'][comp]  + (4/3) * data['MuPDer'][comp] ) - 3 * data['K'][comp] *   \
            ( moduli_der (To, data, x, comp ) ) ) ) 

$$ G = (1-2\epsilon)^{5/2}\left\{G+ \epsilon [5G- 3KG'(T)] \right\}  \dots (eq.13)$$

$$\partial G = (1-2\epsilon)^{5/2}\left\{\frac{\partial G}{\partial T}(T-T_r)+ \epsilon [5\frac{\partial G}{\partial T}(T-T_r)-3\frac{\partial K}{\partial T}  \frac{\partial G}{\partial P}(T-T_r) - 3K\partial G'(T)] \right\}  \dots (eq.14)$$

In [ ]:
# S wave Modulus (P,T)
def s_modulus0 (data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( s_moduliT (data, x, comp)   +   \
            ep * ( 5 * s_moduliT ( data, x, comp)  - 3 * data['K'][comp] * ( smoduli_der0 (data, x, comp ) ) ) )  

In [ ]:
# function for K and mu
def s_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( s_moduli ( To, data, x, comp)   +   \
            ep * ( 5 * s_moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * (  data['MuPDer'][comp] ) - 3 * data['K'][comp] * (3/4) * \
            ( moduli_der (To, data, x, comp ) ) ) )  

In [ ]:
def density_der(comp, rho, To, data, x, ep):
    return - data['rho'][comp] * np.exp (- data['a0'][comp] * (x - 293) - \
        ( data['a1'][comp] / 2) * (x - 293) ** 2  ) * ( data['a0'][comp] + data['a1'][comp] * x ) * \
        ( x - To) * ( 1 - 2*ep )**(3/2.) 

In [ ]:
# inversion for temperature
def double_derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) + f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) -             2 * f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) ) / (h**2)  

def derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) - f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo)  ) / (2 * h)

$$\partial V_p / \partial T=  \partial V_{panh} / \partial T +\partial V_{pane} / \partial T \dots (eq.11)$$

$$ where, $$


$$Q_p^{-1} = \left (1-4/3 \left (\frac{V_s}{V_p} \right )^2 \right )Q_k^{-1} + 4/3 \left (\frac{V_s}{V_p} \right )^2 \left ( Aw^a\exp \left (\frac {a(E+PV)}{RT}  \right ) \right )^{-1} \dots (eq.12) $$

$$ \partial V_p = \frac{1}{\sqrt \rho} \frac{1}{2\sqrt {K+4G/3}}\partial (K+4G/3)-\frac {\sqrt {K+4G/3}}{2 \rho^{3/2}} \partial \rho + {Q_p}^-1\frac{aH}{2RT^2 \tan {\frac {\pi a}{2}}}(T-T_r) \dots (eq.13)$$

In [ ]:
def temperature_p ( x, delVp, ep, rho, To, data, P, Vso, Vpo):
    p_modulus0 = K_avg + (4/3)*Muavg     
    return ( 1/(2 * np.sqrt(rho_avg * p_modulus0) ) ) * ( data['conc'][0] *  p_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  p_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            p_modulus ( To, data, x, 2 , ep ) ) - ( ( np.sqrt(p_modulus0)/( 2 * ( (rho_avg) ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H * (4/3) * (Vso/Vpo)**2 ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVp

# def temperature_p ( x, delVp, ep, rho, To, data, P, Vso, Vpo):
# #    p_modulus0 = K_avg + (4/3)*Muavg     
#     return ( 1/(2 * np.sqrt(rho * ( data['conc'][0] *  p_modulus0 (data, x, 0 , ep ) +
#             data['conc'][1] *  p_modulus0 (data, x, 1 , ep ) + data['conc'][2] *
#             p_modulus0 (data, x, 2 , ep ) )) ) ) * ( data['conc'][0] *  p_modulus ( To, data, x, 0 , ep ) +
#             data['conc'][1] *  p_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
#             p_modulus ( To, data, x, 2 , ep ) ) - ( ( np.sqrt(( data['conc'][0] *  p_modulus0 (data, x, 0 , ep ) +
#             data['conc'][1] *  p_modulus0 (data, x, 1 , ep ) + data['conc'][2] *
#             p_modulus0 (data, x, 2 , ep ) ))/( 2 * ( (rho) ** 3/2 ) ) ) * \
#             ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
#              density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
#              + ( ( a * H * (4/3) * (Vso/Vpo)**2 ) * ( ( A * ( ( 2*np.pi )**a ) * 
#              np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
#              * (x - To) - delVp

### The last equation of appendix A is can be expressed in delivative form like eq. 11
### I think the first term to the third term in eq. 11 are corresponding to the first term to the third term in eq. 12
### , respectively. I understood first term (it is what we are done above) but I don't get it second and third.
### Please share your derivation.

$$\partial V_s / \partial T=  \partial V_{sanh} / \partial T +\partial V_{sane} / \partial T \dots (eq.14)$$

$$ where, $$

$$Q_s^{-1} = \left ( Aw^a\exp \left (\frac {a(E+PV)}{RT}  \right ) \right )^{-1} \dots (eq.15) $$

$$ \partial V_s = \frac{1}{\sqrt \rho} \frac{1}{2\sqrt G}\partial G-\frac {\sqrt G}{2 \rho^{3/2}} \partial \rho + {Q_s}^-1\frac{aH}{2RT^2 \tan {\frac {\pi a}{2}}}(T-T_r) \dots (eq.16)$$

In [ ]:
def temperature_s ( x, delVs, ep, rho, To, data, P, Vso, Vpo):
#    s_modulus0 = Muavg  
    data['KPDer'][:] = 0 ;
    return ( 1/(2 * np.sqrt(rho * (data['conc'][0] * ( s_modulus0 (data, x, 0 , ep ) +
            data['conc'][1] *  s_modulus0 (data, x, 1 , ep ) + data['conc'][2] *
            s_modulus0 (data, x, 2 , ep ) ) )) ) ) * (data['conc'][0] * ( s_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  s_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            s_modulus ( To, data, x, 2 , ep ) ) ) - ( ( np.sqrt((data['conc'][0] * ( s_modulus0 (data, x, 0 , ep ) +
            data['conc'][1] *  s_modulus0 (data, x, 1 , ep ) + data['conc'][2] *
            s_modulus0 (data, x, 2 , ep ) ) ))/( 2 * ( (rho) ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVs

In [ ]:
# def temperature_s ( x, delVs, ep, rho, To, data, P, Vso, Vpo):
#     s_modulus0 = Muavg  
#     data['KPDer'][:] = 0 ;
#     return ( 1/(2 * np.sqrt(rho_avg * s_modulus0) ) ) * (data['conc'][0] * ( s_modulus ( To, data, x, 0 , ep ) +
#             data['conc'][1] *  s_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
#             s_modulus ( To, data, x, 2 , ep ) ) ) - ( ( np.sqrt(s_modulus0)/( 2 * ( (rho_avg) ** 3/2 ) ) ) * \
#             ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
#              density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
#              + ( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
#              np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
#              * (x - To) - delVs

In [ ]:
def abstemp_s ( x, delVs, ep, rho, To, data, P, Vso, Vpo):
    data['KPDer'][:] = 0 ;
    return np.sqrt((data['conc'][0] * s_modulus0 (data, x, 0 , ep )+data['conc'][1] * s_modulus0 (data, x, 1 , ep )+ 
                    data['conc'][2] * s_modulus0 (data, x, 2 , ep ))/((( data['conc'][0] *  density ( data['a0'][0], data['a1'][0], data['a2'][0], data['rho'][0], x) ) 
                + ( data['conc'][1] *  density ( data['a0'][1], data['a1'][1], data['a2'][1], data['rho'][1], x) )
                + ( data['conc'][2] *  density ( data['a0'][2], data['a1'][2], data['a2'][2], data['rho'][2], x) ) ) \
                * ( ( 1 - ( 2 * ep ) ) ** (3/2) )))*(1-( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ))-Vso

In [ ]:
def solve_T (f, x0, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    lastX = x0
    nextX = lastX +  10 * h  # "different than lastX so loop starts OK
    while (abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()        
        newY  = f (nextX, delVp, ep, rho, To, data, P, Vso, Vpo)          # just for debug... see what happens
#         print (newY)     # print out progress... again just debug
        lastX = nextX
        nextX = lastX -  f (nextX,  delVp, ep, rho, To, data, P, Vso, Vpo) / derivative(f, lastX, h,  delVp, ep, rho, To, data, P, Vso, Vpo)  # update estimate using N-R
    return nextX

In [ ]:
To = np.ones(len(depth)) ; ep_cal = np.ones(len(depth))
rho_cal = np.ones(len(depth)); T = np.ones(len(depth)) 
P = np.ones(len(depth)) ; AT = np.ones(len(depth)) 

In [ ]:
# invert for temperatures
x0 = 500;
Ax0= 500;
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
T_a = adiabat[:,2]
Vp_o  = data_korean[:,4]*1.e3 # unit chang km into m
Vs_o  = data_korean[:,6]*1.e3 # unit chang km into m

for i in range(len(depth)):
#    data['a2'][:] = 0; # ignore second terms
    To[i]   = (T_ref[:,1][T_ref[:,0] <= depth[i]].max()) # + 273# reference temperature with depth
    ep_cal[i]  = strain [depth_a <= depth[i]].max()
    P[i]     = (pressures[depth_a <= depth[i]].max())
    rho_cal[i] = rho [depth_a <= depth[i]].max()
    
#    print(depth[i], Vs_o[i], P[i], rho_cal[i])
#     # equation to get x
   # T[i]     = solve_T (temperature_s, x0, 1,  S_data_anonmal[i]*1e3, ep_cal[i], rho_cal[i], To[i], data, P[i], Vs_o[i], Vp_o[i])
    AT[i]    = solve_T (abstemp_s,    Ax0, 1,  S_data_anonmal[i]*1e3, ep_cal[i], rho_cal[i], To[i], data, P[i], Vs_o[i], Vp_o[i])

In [ ]:
T_results = np.loadtxt('initial_temperature_S_tomography.txt', skiprows=4, usecols=3)

In [ ]:
d_crust = (deep < 20.0) & (deep > 0.0)
lat_c = data_korean[d_crust, 1]
long_c = data_korean[d_crust, 0]
d_c = data_korean[d_crust, 2]
T_crust = ((data_korean[d_crust,4] - data_korean[d_crust,6]) * grad_crust) + 273

d_crust = (deep < 40.0) & (deep > 20.0)
lat_c = data_korean[d_crust, 1]
long_c = data_korean[d_crust, 0]
d_c = data_korean[d_crust, 2]
T_crust = ((data_korean[d_crust,4] - data_korean[d_crust,6]) * grad_crust2) + 473

In [ ]:
print (d_c)

In [ ]:
d_q = np.unique(depth)
d_cal   = np.reshape(depth,(len(d_q), 101, 95))
#T_cal   = np.reshape(T, (len(d_q), 101, 95))
T_cal   = np.reshape(AT, (len(d_q), 101, 95))
dVs_cal = np.reshape(S_data_anonmal , (len(d_q), 101, 95))
# Vp_cal = np.reshape(Vp, (len(d_q), npts, npts))
lat_cal = np.reshape(latitude,(len(d_q), 101, 95))
long_cal= np.reshape(longitude,(len(d_q), 101, 95))
n = 13
(fig, (ax1, ax2)) = plt.subplots(1, 2, figsize=(20,6))
plt.rcParams.update({'font.size': 14})
CB = ax1.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], T_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
CB1 = ax2.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], dVs_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
cbar = plt.colorbar(CB, shrink=0.6, ax=ax1)
cbar2 = plt.colorbar(CB1, shrink=0.6, ax=ax2)

CB.axes.set_xlim([122, 135])
CB.axes.set_ylim([31, 41])
CB1.axes.set_xlim([122, 135])
CB1.axes.set_ylim([31, 41])
plt.show()
print(np.unique(depth)[n])
print(np.unique(depth))

In [ ]:
print (d_q[14])

In [ ]:
# addting a buffer layer
T_c = np.concatenate([ T, T_crust]);
T_a = np.concatenate([ AT, T_crust]);
lat = np.concatenate([latitude, lat_c])
longt = np.concatenate([longitude, long_c])
depth_c = np.concatenate([depth, d_c])
print (np.shape(lat ), np.shape(depth_c), np.shape(T_c))

r = (6370 - depth_c)*1e3
long_f = longt*np.pi/180
lat_f = (90 - lat)*np.pi/180
ind = np.lexsort((r, long_f, lat_f))
np.savetxt('initial_temperature_S_tomography.txt', \
            np.stack((r[ind], long_f[ind], lat_f[ind], T_a[ind]), axis=-1))

In [ ]:
min(T)
# print([S_data_anonmal == max(S_data_anonmal)])

In [ ]:
print(long_f[ind])
